In [128]:
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 280

import matplotlib.pyplot as plt

In [114]:
dataFolder_path = '../../data/'

In [115]:
data_df = pd.read_csv(dataFolder_path+'judge_1377884607_tweet_product_company.csv')

In [129]:
def dataFrame_info(df):
    '''
    more verbose version of df.info()
    '''
    df_info_dict = {}
    for col in data_df.columns:
        num_zeroes = (df[col] == 0).sum()
        num_nulls = df[col].isna().sum()
        num_uniques = len(df[col].unique())

        mean = 0
        median = 0

        list_missingOrUnknown = ['unknown', ' ', '', 'missing']

        try:
            num_missingOrUnknown = (df[col].map(
                lambda x: x.lower() in list_missingOrUnknown).sum())
        except:
            num_missingOrUnknown = 0

        try:
            mean = df[col].mean()
            median = df[col].median()
        except:
            mean = 0
            median = 0

        df_info_dict[col] = {'Zeroes': num_zeroes,
                             'Nulls': num_nulls,
                             'Uniques': num_uniques,
                             'Missing/Unknown': num_missingOrUnknown,
                             'Mean': mean,
                             'Median': median}
    df_pd = pd.DataFrame(
              index=pd.Index(df.columns, name='Columns:'),
              columns=pd.MultiIndex.from_product([['Zeroes', 'Nulls', 'Uniques','Missing/Unknown'],['Count', 'Fraction']], 
                                                 names=['Info Table:', 'Details:']))
    df_pd[['Mean','Median']] = np.nan

    for col, info in df_info_dict.items():
        df_pd.loc[col,('Zeroes','Count')] = info['Zeroes']
        df_pd.loc[col,('Zeroes','Fraction')] = "{:.2f} %".format((info['Zeroes'] / df.shape[0]) * 100)
        
        df_pd.loc[col,('Nulls','Count')] = info['Nulls']
        df_pd.loc[col,('Nulls','Fraction')] = "{:.2f} %".format((info['Nulls'] / df.shape[0]) * 100)
        
        df_pd.loc[col,('Uniques','Count')] = info['Uniques']
        df_pd.loc[col,('Uniques','Fraction')] = "{:.2f} %".format((info['Uniques'] / df.shape[0]) * 100)
        
        df_pd.loc[col,('Missing/Unknown','Count')] = info['Missing/Unknown']
        df_pd.loc[col,('Missing/Unknown','Fraction')] = "{:.2f} %".format((info['Missing/Unknown'] / df.shape[0]) * 100)
        
        df_pd.loc[col,'Mean'] = info['Mean']
        df_pd.loc[col,'Median'] = info['Median']
        
            
    display(df_pd)

In [130]:
data_df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive emotion


In [131]:
dataFrame_info(data_df)

Info Table:                                        Zeroes          Nulls  \
Details:                                            Count Fraction Count   
Columns:                                                                   
tweet_text                                              0   0.00 %     0   
emotion_in_tweet_is_directed_at                         0   0.00 %  5551   
is_there_an_emotion_directed_at_a_brand_or_product      0   0.00 %     0   

Info Table:                                                 Uniques           \
Details:                                           Fraction   Count Fraction   
Columns:                                                                       
tweet_text                                           0.00 %    8693  99.69 %   
emotion_in_tweet_is_directed_at                     63.66 %      10   0.11 %   
is_there_an_emotion_directed_at_a_brand_or_product   0.00 %       4   0.05 %   

Info Table:                                        Missing/Unknown           \
Details:                                                     Count Fraction   
Columns:                                                                      
tweet_text                                                       0   0.00 %   
emotion_in_tweet_is_directed_at                                  0   0.00 %   
is_there_an_emotion_directed_at_a_brand_or_product               0   0.00 %   

Info Table:                                        Mean Median  
Details:                                                        
Columns:                                                        
tweet_text                                          0.0    0.0  
emotion_in_tweet_is_directed_at                     0.0    0.0  
is_there_an_emotion_directed_at_a_brand_or_product  0.0    0.0

In [133]:
data_df[data_df.tweet_text.isna()]

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product


In [134]:
data_df.dropna(subset=['tweet_text'],inplace=True)

In [135]:
data_df.shape

(8720, 3)

In [137]:
data_df[data_df.emotion_in_tweet_is_directed_at.isna()]

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
5,@teachntech00 New iPad Apps For #SpeechTherapy And Communication Are Showcased At The #SXSW Conference http://ht.ly/49n4M #iear #edchat #asd,NaN,No emotion toward brand or product
16,Holler Gram for iPad on the iTunes App Store - http://t.co/kfN3f5Q (via @marc_is_ken) #sxsw,NaN,No emotion toward brand or product
32,"Attn: All #SXSW frineds, @mention Register for #GDGTLive and see Cobra iRadar for Android. {link}",NaN,No emotion toward brand or product
33,Anyone at #sxsw want to sell their old iPad?,NaN,No emotion toward brand or product
34,Anyone at #SXSW who bought the new iPad want to sell their older iPad to me?,NaN,No emotion toward brand or product
...,...,...,...
8715,"@mention Yup, but I don't have a third app yet. I'm on Android, any suggestions? #SXSW CC: @mention",NaN,No emotion toward brand or product
8717,"Wave, buzz... RT @mention We interrupt your regularly scheduled #sxsw geek programming with big news {link} #google #circles",NaN,No emotion toward brand or product
8718,"Google's Zeiger, a physician never reported potential AE. Yet FDA relies on physicians. &quot;We're operating w/out data.&quot; #sxsw #health2dev",NaN,No emotion toward brand or product
8719,Some Verizon iPhone customers complained their time fell back an hour this weekend. Of course they were the New Yorkers who attended #SXSW.,NaN,No emotion toward brand or product


In [151]:
data_df.tweet_text.str.split()

0                      [.@wesley83, I, have, a, 3G, iPhone., After, 3, hrs, tweeting, at, #RISE_Austin,, it, was, dead!, I, need, to, upgrade., Plugin, stations, at, #SXSW.]
1          [@jessedee, Know, about, @fludapp, ?, Awesome, iPad/iPhone, app, that, you'll, likely, appreciate, for, its, design., Also,, they're, giving, free, Ts, at, #SXSW]
2                                                                             [@swonderlin, Can, not, wait, for, #iPad, 2, also., They, should, sale, them, down, at, #SXSW.]
3                                                                          [@sxsw, I, hope, this, year's, festival, isn't, as, crashy, as, this, year's, iPhone, app., #sxsw]
4                       [@sxtxstate, great, stuff, on, Fri, #SXSW:, Marissa, Mayer, (Google),, Tim, O'Reilly, (tech, books/conferences), &amp;, Matt, Mullenweg, (Wordpress)]
                                                                                        ...                                       

In [144]:
data_df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive emotion
...,...,...,...
8716,Ipad everywhere. #SXSW {link},iPad,Positive emotion
8717,"Wave, buzz... RT @mention We interrupt your regularly scheduled #sxsw geek programming with big news {link} #google #circles",NaN,No emotion toward brand or product
8718,"Google's Zeiger, a physician never reported potential AE. Yet FDA relies on physicians. &quot;We're operating w/out data.&quot; #sxsw #health2dev",NaN,No emotion toward brand or product
8719,Some Verizon iPhone customers complained their time fell back an hour this weekend. Of course they were the New Yorkers who attended #SXSW.,NaN,No emotion toward brand or product
